In [1]:
library(Seurat)
library(dplyr)
library(tidyr)
library(RColorBrewer)
library("ggsci")
library(ggplot2)
library(presto)
library(JASPAR2020)
library(TFBSTools)
library(motifmatchr)

载入需要的程序包：SeuratObject

载入需要的程序包：sp


载入程序包：‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t



载入程序包：‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


载入需要的程序包：Rcpp

载入需要的程序包：data.table


载入程序包：‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




In [2]:
packageVersion("Seurat")

[1] ‘5.3.0’

In [2]:
run_motif_de_all_pairs <- function(z_rds_path, seurat_rds_path) {
  # 1. 读取数据
  data <- readRDS(z_rds_path)
  seurat_obj_orig <- readRDS(seurat_rds_path)
  
  cell_info <- data.frame(
    barcode = rownames(seurat_obj_orig@meta.data),
    BMI_group = seurat_obj_orig@meta.data$BMI_group,
    stringsAsFactors = FALSE
  )
  
  
na_per_gene <- rowSums(is.na(data))
motifs_to_keep <- names(na_per_gene)[na_per_gene == 0] 
data_subset <- data[motifs_to_keep, , drop = FALSE]
  
  # 5. 创建 Seurat 对象
  seurat_sub <- CreateSeuratObject(
    counts = data_subset,
    meta.data = cell_info,
    assay = "RNA"
  )
  
  # 6. 赋值到 @data
  counts_mat <- GetAssayData(seurat_sub, assay = "RNA", layer = "counts")
  rownames(counts_mat) <- rownames(seurat_sub[["RNA"]]@features)
  colnames(counts_mat) <- rownames(seurat_sub[["RNA"]]@cells)
  
  seurat_sub <- SetAssayData(
    object = seurat_sub,
    assay = "RNA",
    slot = "data",
    new.data = counts_mat
  )
  
  # 7. 设置 identity
  seurat_sub <- SetIdent(seurat_sub, value = "BMI_group")
  
  # 检查实际存在的组
  present_groups <- levels(Idents(seurat_sub))
  cat("Present groups:", paste(present_groups, collapse = ", "), "\n")
  
  # 8. 生成所有两两组合（无重复，不考虑顺序）
  group_pairs <- t(combn(present_groups, 2))
  
  if (nrow(group_pairs) == 0) {
    warning("Less than 2 groups present. Cannot do pairwise comparison.")
    return(NULL)
  }
  
  # 9. 对每对运行 FindMarkers
  all_results <- list()
  
  for (i in 1:nrow(group_pairs)) {
    g1 <- group_pairs[i, 1]
    g2 <- group_pairs[i, 2]
    
    cat("  Comparing:", g1, "vs", g2, "\n")
    
    deg <- FindMarkers(
      object = seurat_sub,
      ident.1 = g1,
      ident.2 = g2,
      assay = "RNA",
      slot = "data",
      test.use = "wilcox",
      mean.fxn = rowMeans,
      logfc.threshold = 0,
      min.pct = 0.01
    )
    
    # 添加元信息
    deg$motif <- rownames(deg)
    deg <- deg %>%
      mutate(comp = g1, control = g2, comparison = paste0(g1, "_vs_", g2))
    
    # 保存显著结果
    deg_sig <- deg %>% filter(p_val_adj < 0.05)
    all_results[[paste0(g1, "_vs_", g2)]] <- deg_sig
  }
  
  # 10. 合并所有结果
  if (length(all_results) > 0) {
    final_df <- bind_rows(all_results)
  } else {
    final_df <- NULL
  }
  
  return(final_df)
}

In [3]:
seurat_dir <- "/media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/seurat/"
z_dir      <- "/media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/motif_chromVAR/output/z_matrix/"

# 获取文件名（不含路径）
seurat_files <- list.files(seurat_dir, pattern = "\\.rds$", full.names = FALSE)
z_files      <- list.files(z_dir,      pattern = "\\.rds$", full.names = FALSE)

# 提取基础名（去掉 .rds）
seurat_basenames <- tools::file_path_sans_ext(seurat_files)   # e.g. "MAIT_SLC4A10"
z_basenames      <- tools::file_path_sans_ext(z_files)        # e.g. "MAIT_SLC4A10_z"

# 从 z_basenames 中移除末尾的 "_z"
z_cleaned <- sub("_z$", "", z_basenames)  # "MAIT_SLC4A10_z" → "MAIT_SLC4A10"

# 找出共同的细胞类型（即 cleaned z 名 在 seurat 中存在）
common <- intersect(seurat_basenames, z_cleaned)

cat("Found", length(common), "common cell types:\n")
print(common)

# 构建匹配对
results_list <- vector("list", length(common))
all_results_list <- list() 

Found 64 common cell types:
 [1] "AS_DC"                      "Atypical_Bm_ITGAX"         
 [3] "Bn_IFIT3"                   "Bn_TCL1A"                  
 [5] "CD4_CTL_GZMH"               "CD4_Tcm_CXCR5"             
 [7] "CD4_Tcm_IFI44L"             "CD4_Tem_CCR5"              
 [9] "CD4_Tem_CCR7neg"            "CD4_Tfh-like_CXCR5"        
[11] "CD4_Th_CCR4"                "CD4_Th_LMNA"               
[13] "CD4_Th_TNFRSF11A"           "CD4_Th17-like_GZMK"        
[15] "CD4_Th17-like_RORC"         "CD4_Th22-like_CCR10"       
[17] "CD4_Tn_CCR7"                "CD4_Tr1_IL10"              
[19] "CD4_Treg_FCRL3"             "CD4_Treg_FOXP3"            
[21] "CD8_CTL_GZMB"               "CD8_CTL_GZMK"              
[23] "CD8_CTL_IFI44L"             "CD8_Tcm_IFI44L"            
[25] "CD8_Tem_CCR7neg"            "CD8_Tn_CCR7"               
[27] "cMono_CD14"                 "cMono_CXCL10"              
[29] "cMono_IFI44L"               "cMono_IL1B"                
[31] "DC_CSF2RA"           

In [4]:
for (i in seq_along(common)) {
  ct <- common[i]
  
  seurat_path <- file.path(seurat_dir, paste0(ct, ".rds"))
  z_path      <- file.path(z_dir,      paste0(ct, "_z.rds"))
  
  cat("\n>>> Processing:", ct, "\n")
  
  res <- tryCatch({
    run_motif_de_all_pairs(z_rds_path = z_path, seurat_rds_path = seurat_path)
  }, error = function(e) {
    warning("Error in ", ct, ": ", e$message)
    return(NULL)
  })
  
  if (!is.null(res) && nrow(res) > 0) {
    res$cell_type <- ct
  } else {
    res <- NULL  # 明确标记为空
  }
  all_results_list[[i]] <- res
}

# 找出哪些是空的（NULL 或 0 行）
empty_cts <- names(all_results_list)[sapply(all_results_list, function(x) is.null(x) || nrow(x) == 0)]

if (length(empty_cts) > 0) {
  cat("The following cell types produced empty or failed results:\n")
  print(empty_cts)
} else {
  cat("All cell types produced valid results.\n")
}

# 过滤有效结果
valid_results <- all_results_list[!sapply(all_results_list, function(x) is.null(x) || nrow(x) == 0)]

if (length(valid_results) == 0) {
  stop("No valid results.")
}


>>> Processing: AS_DC 


Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
Warning message:
“The `slot` argument of `SetAssayData()` is deprecated as of SeuratObject 5.0.0.
ℹ Please use the `layer` argument instead.”


Present groups: Overweight, Underweight, normal 
  Comparing: Overweight vs Underweight 


Warning message:
“The `slot` argument of `GetAssayData()` is deprecated as of SeuratObject 5.0.0.
ℹ Please use the `layer` argument instead.
ℹ The deprecated feature was likely used in the Seurat package.
  Please report the issue at <https://github.com/satijalab/seurat/issues>.”
Warning message:
“`PackageCheck()` was deprecated in SeuratObject 5.0.0.
ℹ Please use `rlang::check_installed()` instead.
ℹ The deprecated feature was likely used in the Seurat package.
  Please report the issue at <https://github.com/satijalab/seurat/issues>.”


  Comparing: Overweight vs normal 
  Comparing: Underweight vs normal 

>>> Processing: Atypical_Bm_ITGAX 


...

Present groups: Overweight, Underweight, normal 
  Comparing: Overweight vs Underweight 
  Comparing: Overweight vs normal 
  Comparing: Underweight vs normal 

>>> Processing: Unswitched_Bm_IL6 


Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


Present groups: normal 


Warning message in value[[3L]](cond):
“Error in Unswitched_Bm_IL6: 在为函数“t”选择方法时计算参数“x”时出错：n < m”



✅ All cell types produced valid results.


In [5]:
combined_results <- bind_rows(valid_results)
rownames(combined_results) <- NULL

In [6]:
combined_results

p_val,avg_log2FC,pct.1,pct.2,p_val_adj,motif,comp,control,comparison,cell_type
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
9.742530e-06,-0.14289827,0.452,0.512,6.167021e-03,MA0037.3,Underweight,normal,Underweight_vs_normal,Atypical_Bm_ITGAX
5.401176e-09,0.07790191,0.522,0.486,3.418944e-06,MA0081.2,Overweight,Underweight,Overweight_vs_Underweight,Bn_TCL1A
1.752601e-07,0.06661647,0.528,0.496,1.109396e-04,MA0080.5,Overweight,Underweight,Overweight_vs_Underweight,Bn_TCL1A
1.028906e-06,0.06873111,0.523,0.491,6.512976e-04,MA0651.1,Overweight,Underweight,Overweight_vs_Underweight,Bn_TCL1A
1.432613e-06,-0.06608531,0.471,0.501,9.068440e-04,MA1121.1,Overweight,Underweight,Overweight_vs_Underweight,Bn_TCL1A
1.618824e-06,-0.06692834,0.471,0.504,1.024716e-03,MA1475.1,Overweight,Underweight,Overweight_vs_Underweight,Bn_TCL1A
2.433868e-06,0.07318936,0.497,0.472,1.540639e-03,MA0864.2,Overweight,Underweight,Overweight_vs_Underweight,Bn_TCL1A
2.527994e-06,0.06777198,0.521,0.492,1.600220e-03,MA0598.3,Overweight,Underweight,Overweight_vs_Underweight,Bn_TCL1A
2.646313e-06,0.06063479,0.520,0.489,1.675116e-03,MA1631.1,Overweight,Underweight,Overweight_vs_Underweight,Bn_TCL1A


In [7]:
#获取motif对应TF的name
pwm <- getMatrixSet(
  x = JASPAR2020,
  opts = list(species = 9606, all =FALSE ,factorType = "PFM")#似乎需要修改
)

name = sapply(pwm, function(x) x@name)

metadata <- data.frame(
  jaspar_id = names(pwm),           # MA0030.1, MA0031.1  
  name = sapply(pwm, function(x) x@name),            # MAF, MYC
  stringsAsFactors = FALSE
)

# 去重（因为 MA0030.1 和 MA0030.2 都对应 MA0030）
unique_mapping <- unique(metadata[c("jaspar_id", "name")])

In [8]:
combined_results <- combined_results %>%
  left_join(
    unique_mapping,
    by = c("motif" = "jaspar_id")
  )

In [9]:
combined_results

p_val,avg_log2FC,pct.1,pct.2,p_val_adj,motif,comp,control,comparison,cell_type,name
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
9.742530e-06,-0.14289827,0.452,0.512,6.167021e-03,MA0037.3,Underweight,normal,Underweight_vs_normal,Atypical_Bm_ITGAX,GATA3
5.401176e-09,0.07790191,0.522,0.486,3.418944e-06,MA0081.2,Overweight,Underweight,Overweight_vs_Underweight,Bn_TCL1A,SPIB
1.752601e-07,0.06661647,0.528,0.496,1.109396e-04,MA0080.5,Overweight,Underweight,Overweight_vs_Underweight,Bn_TCL1A,SPI1
1.028906e-06,0.06873111,0.523,0.491,6.512976e-04,MA0651.1,Overweight,Underweight,Overweight_vs_Underweight,Bn_TCL1A,HOXC11
1.432613e-06,-0.06608531,0.471,0.501,9.068440e-04,MA1121.1,Overweight,Underweight,Overweight_vs_Underweight,Bn_TCL1A,TEAD2
1.618824e-06,-0.06692834,0.471,0.504,1.024716e-03,MA1475.1,Overweight,Underweight,Overweight_vs_Underweight,Bn_TCL1A,CREB3L4(var.2)
2.433868e-06,0.07318936,0.497,0.472,1.540639e-03,MA0864.2,Overweight,Underweight,Overweight_vs_Underweight,Bn_TCL1A,E2F2
2.527994e-06,0.06777198,0.521,0.492,1.600220e-03,MA0598.3,Overweight,Underweight,Overweight_vs_Underweight,Bn_TCL1A,EHF
2.646313e-06,0.06063479,0.520,0.489,1.675116e-03,MA1631.1,Overweight,Underweight,Overweight_vs_Underweight,Bn_TCL1A,ASCL1(var.2)


In [81]:
combined_results <- merge(combined_results, motif_name_map, by = "motif", all.x = TRUE)

# 可选：检查是否有 motif 在 dev 中找不到对应 name
missing_motifs <- combined_results[is.na(combined_results$name), "motif"]
if (length(missing_motifs) > 0) {
  cat("Warning: 以下 motif 在 dev 中未找到对应的 name（已设为 NA）:\n")
  print(unique(missing_motifs))
}

In [12]:
write.csv(combined_results, file = "combined_diff_acc_sig_findmarker_20251219.csv", row.names = FALSE)